In [1]:
import requests
import tarfile
from os import path

In [2]:
# Downloading the dataset

fname = 'airline_2m.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)


151681776

In [3]:
# Extracting the dataset
tar = tarfile.open(fname)
tar.extractall()
tar.close()


In [2]:
# Verifying the file was extracted properly
data_path = "airline_2m.csv"
path.exists(data_path)

True

In [3]:
import pandas as pd 

In [4]:
df = pd.read_csv(data_path, encoding = "ISO-8859-1",
                 dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str})

In [5]:
df.shape

(2000000, 109)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Columns: 109 entries, Year to Div5TailNum
dtypes: float64(72), int64(18), object(19)
memory usage: 1.6+ GB


In [ ]:
df1=pd.DataFrame()
df1['Unique']=pd.DataFrame(df.nunique())
df1['Count']=pd.DataFrame(df.count())
df1=df1.rename(columns={0: 'Unique'})
df1.reset_index()
df1

,Unique,Count
Year,34,2000000
Quarter,4,2000000
Month,12,2000000
DayofMonth,31,2000000
DayOfWeek,7,2000000
...,...,...
Div5WheelsOn,0,0
Div5TotalGTime,0,0
Div5LongestGTime,0,0
Div5WheelsOff,0,0


In [ ]:
#df1.to_csv('cases.csv')

In [ ]:
df1=pd.read_csv('cases.csv')
df1

,Column,Unique,Count,Categorical?,Needed?
0,Year,34,2000000,1.0,1.0
1,Quarter,4,2000000,1.0,1.0
2,Month,12,2000000,1.0,1.0
3,DayofMonth,31,2000000,1.0,1.0
4,DayOfWeek,7,2000000,1.0,1.0
5,DOT_ID_Reporting_Airline,34,2000000,1.0,1.0
6,Flight_Number_Reporting_Airline,8050,2000000,1.0,1.0
7,OriginAirportID,407,2000000,1.0,1.0
8,OriginState,53,1999354,1.0,1.0
9,DestAirportID,409,2000000,1.0,1.0


In [ ]:
featurecols= pd.Series(df1['Column'])
featurecols

0                                Year
1                             Quarter
2                               Month
3                          DayofMonth
4                           DayOfWeek
5            DOT_ID_Reporting_Airline
6     Flight_Number_Reporting_Airline
7                     OriginAirportID
8                         OriginState
9                       DestAirportID
10                          DestState
11                            DepTime
12                          Cancelled
Name: Column, dtype: object

In [6]:
featurecols= ['Year','Quarter',
'Month','DayofMonth','DayOfWeek','DOT_ID_Reporting_Airline','Flight_Number_Reporting_Airline',
'OriginAirportID','OriginState','DestAirportID','DestState','DepTime','Cancelled']
featurecols

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DOT_ID_Reporting_Airline',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginState',
 'DestAirportID',
 'DestState',
 'DepTime',
 'Cancelled']

In [7]:
featuredata=df[featurecols]
featuredata.to_csv('featuredata.csv',index=False)

In [15]:
featuredata=pd.read_csv('featuredata.csv')
featuredata=featuredata.dropna()
featuredata

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime,Cancelled
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659.0,0.0
1,2009,2,5,28,4,20437,671,13342,WI,13204,FL,1202.0,0.0
2,2013,2,6,29,6,20398,3297,11921,CO,11298,TX,1644.0,0.0
3,2010,3,8,31,2,19790,1806,12892,CA,11433,MI,1305.0,0.0
4,2006,1,1,15,7,20355,465,11618,NJ,11057,NC,1911.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,2008,1,3,23,7,19393,966,12889,NV,14107,AZ,1444.0,0.0
1999996,1999,1,1,5,2,19704,529,11618,NJ,11298,TX,945.0,0.0
1999997,2003,4,11,14,5,20355,1457,10994,SC,11057,NC,1219.0,0.0
1999998,2012,2,5,15,2,19393,536,13232,IL,10693,TN,1838.0,0.0


In [16]:
target_name = "Cancelled"
target = featuredata[target_name]

data = featuredata.drop(columns=[target_name])

In [17]:
target.value_counts()

0.0    1962513
1.0        457
Name: Cancelled, dtype: int64

In [18]:
data.isnull().values.any()

False

In [19]:
data

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659.0
1,2009,2,5,28,4,20437,671,13342,WI,13204,FL,1202.0
2,2013,2,6,29,6,20398,3297,11921,CO,11298,TX,1644.0
3,2010,3,8,31,2,19790,1806,12892,CA,11433,MI,1305.0
4,2006,1,1,15,7,20355,465,11618,NJ,11057,NC,1911.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,2008,1,3,23,7,19393,966,12889,NV,14107,AZ,1444.0
1999996,1999,1,1,5,2,19704,529,11618,NJ,11298,TX,945.0
1999997,2003,4,11,14,5,20355,1457,10994,SC,11057,NC,1219.0
1999998,2012,2,5,15,2,19393,536,13232,IL,10693,TN,1838.0


In [29]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(data, target)
dummy_preds=dummy_clf.predict(data)
dummy_clf.predict(data)
dummy_clf.score(data, target)

0.9997671895138489

In [21]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

logistic_regression = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore'), LogisticRegression()
)


from sklearn.model_selection import cross_validate
cv_results = cross_validate(
    logistic_regression, data, target, cv=10,return_estimator=True)

cv_results

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


{'estimator': (Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype=<class 'numpy.float64'>,
                                 handle_unknown='ignore', sparse=True)),
                  ('logisticregression',
                   LogisticRegression(C=1.0, class_weight=None, dual=False,
                                      fit_intercept=True, intercept_scaling=1,
                                      l1_ratio=None, max_iter=100,
                                      multi_class='auto', n_jobs=None,
                                      penalty='l2', random_state=None,
                                      solver='lbfgs', tol=0.0001, verbose=0,
                                      warm_start=False))],
           verbose=False), Pipeline(memory=None,
           steps=[('onehotencoder',
                   OneHotEncoder(categories='auto', drop=None,
                                 dtype

In [22]:
array=cv_results["test_score"]
array.mean()

0.999767189513849

In [23]:
logistic_regression.fit(data,target)
predictions=logistic_regression.predict(data.iloc[0:1])
predictions

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


array([0.])

In [ ]:
data.iloc[0:1]

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginState,DestAirportID,DestState,DepTime
0,1998,1,1,2,5,19386,675,13487,MN,14869,UT,1659.0


In [24]:
predictions=logistic_regression.predict(data)
predictions

array([0., 0., 0., ..., 0., 0., 0.])

In [26]:
from sklearn.metrics import confusion_matrix
c_matrix=confusion_matrix(target, predictions)
c_matrix

array([[1962513,       0],
       [    457,       0]])

In [32]:
tn, fp, fn, tp = c_matrix.ravel()
(tn, fp, fn, tp)

(1962513, 0, 457, 0)

In [31]:
c_matrix_dummy=confusion_matrix(target, dummy_preds)
tn, fp, fn, tp = c_matrix_dummy.ravel()
(tn, fp, fn, tp)

(1962513, 0, 457, 0)

In [34]:
c_matrix_=confusion_matrix(predictions, dummy_preds)
c_matrix_


array([[1962970]])

In [39]:
y_true = [1,1,1,1,1,1,1]
y_pred = [1,1,1,1,1,1,1]
confusion_matrix(y_true, y_pred)

array([[7]])

In [40]:
target.value_counts()

0.0    1962513
1.0        457
Name: Cancelled, dtype: int64

Class imbalance is an issue
